In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-09-25"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
1570,2024-09-25,Europa Eurocopa,13:00,BC Wolves,Bahcesehir Kol.,1.86,1.86,166.5,1.85,1.85,-1.5,2.00,1.72,UHa2BswH,0.537634,0.537634,0.540541,0.540541,0.075269,0.8,0.0,NaN,NaN,160.608,114.032176,0.710003,NaN,NaN,NaN,117.538,0.000,115.36,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.106446,-1.31,-0.262,-3.282443,0.628935,0.4,-0.228935,0.00,0.000,inf,0.000000,0.0,0.000000
1571,2024-09-25,Europa Eurocopa,13:00,Cluj-Napoca,Cedevita Olimpija,1.44,2.65,166.5,1.87,1.79,-5.5,1.92,1.78,GnwqOKMM,0.694444,0.377358,0.534759,0.558659,0.071803,0.2,0.4,NaN,NaN,100.792,15.299893,0.151797,217.420,231.781338,1.066053,97.384,244.820,86.52,89.90,0.0,0.0,0.0,0.0,0.418386,0.030912,0.053511,0.63,0.126,3.492063,0.868505,0.9,0.031495,-1.21,-0.242,-6.818182,0.429491,0.2,-0.229491
1572,2024-09-25,Europa Eurocopa,14:00,Besiktas,Buducnost,1.43,2.68,161.5,1.87,1.79,-5.5,1.91,1.79,2qD99L7T,0.699301,0.373134,0.534759,0.558659,0.072435,0.6,0.4,NaN,NaN,149.966,17.854422,0.119056,170.074,50.376329,0.296202,122.036,153.492,136.51,151.59,0.0,0.0,0.0,0.0,0.430114,0.030912,0.045866,-3.81,-0.762,-0.564304,0.000000,0.0,0.000000,3.60,0.720,2.333333,0.634661,0.9,0.265339
1573,2024-09-25,Europa Eurocopa,14:00,Trefl Sopot,Ulm,2.65,1.44,164.5,1.84,1.86,4.5,1.93,1.78,0x8oFPwh,0.377358,0.694444,0.543478,0.537634,0.071803,0.0,0.4,NaN,NaN,NaN,NaN,NaN,183.722,47.060068,0.256148,0.000,218.740,0.00,238.50,0.0,0.0,0.0,0.0,0.418386,0.007644,0.057178,0.00,0.000,inf,0.000000,0.0,0.000000,2.66,0.532,0.827068,0.511751,0.6,0.088249
1574,2024-09-25,Europa Eurocopa,15:00,Venezia,Aris,1.24,3.77,155.5,1.88,1.79,-9.5,1.94,1.77,0KEFIbpp,0.806452,0.265252,0.531915,0.558659,0.071704,0.8,0.6,NaN,NaN,153.960,54.304040,0.352715,180.208,52.066783,0.288926,146.762,277.664,205.20,240.80,0.0,0.0,0.0,0.0,0.714164,0.034681,0.064802,-0.02,-0.004,-60.000000,0.000000,0.0,0.000000,0.92,0.184,15.054348,0.000000,0.0,0.000000
1575,2024-09-25,Alemanha Bbl,15:00,Vechta,Bonn,1.78,2.04,165.5,1.84,1.94,-2.5,2.04,1.76,viudFKrh,0.561798,0.490196,0.543478,0.515464,0.051994,0.2,0.4,NaN,NaN,169.720,54.933944,0.323674,149.974,37.155824,0.247748,148.878,146.512,185.59,143.29,0.0,0.0,0.0,0.0,0.096255,0.037413,0.104205,-3.33,-0.666,-1.171171,0.592536,0.4,-0.192536,2.66,0.532,1.954887,0.629859,0.7,0.070141
1576,2024-09-25,Eslováquia Extraliga,13:00,BKM Lucenec,Handlova,1.72,1.95,156.5,1.83,1.83,-2.5,1.99,1.70,4UlU99gn,0.581395,0.512821,0.546448,0.546448,0.094216,0.6,0.2,NaN,NaN,296.904,270.616580,0.911462,158.844,20.898720,0.131568,178.810,155.768,179.78,146.20,0.0,0.0,0.0,0.0,0.088629,0.000000,0.111144,-1.12,-0.224,-3.214286,0.523471,0.7,0.176529,-3.14,-0.628,-1.512739,0.000000,0.0,0.000000
1577,2024-09-25,Eslováquia Extraliga,13:15,BC Komarno,Levice,3.52,1.24,160.5,1.87,1.79,6.5,1.94,1.73,Cfhx8mOb,0.284091,0.806452,0.534759,0.558659,0.090543,0.2,0.6,NaN,NaN,135.156,31.726735,0.234742,136.468,40.107589,0.293897,140.628,125.724,130.95,197.60,0.0,0.0,0.0,0.0,0.677396,0.030912,0.080922,1.92,0.384,6.562500,0.000000,0.0,0.000000,-0.48,-0.096,-2.500000,0.000000,0.0,0.000000
1578,2024-09-25,Eslovênia Liga Otp Banka,15:15,Krka,Podcetrtek,1.11,6.15,160.5,1.83,1.83,-13.5,2.00,1.72,fV5cs9Ij,0.900901,0.162602,0.546448,0.546448,0.063503,0.4,0.0,NaN,NaN,161.094,21.090393,0.130920,NaN,NaN,NaN,174.118,0.000,191.10,0.00,0.0,0.0,0.0,0.0,0.981768,0.000000,0.106446,-3.33,-0.666,-0.165165,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1579,2024-09-25,Eua Wnba,20:30,

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos
